In [2]:
import numpy as np
import pandas as pd
import theano
import theano.tensor as T

import lasagne

theano/tensor/signal/downsample.py:5: UserWarning: downsample module has been moved to the pool module.
  warnings.warn("downsample module has been moved to the pool module.")


In [15]:
path="data/"
data=pd.read_csv(path+"input_train.csv", index_col="ID").filter(regex= "ACC*")
labels=pd.read_csv(path+"challenge_output_data_training_file_sleep_stages_classification.csv", sep=";")

In [16]:
# Load the dataset
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, labels["TARGET"].values, test_size=0.10, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.10, random_state=42)

In [17]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [18]:
X_train=X_train.reshape(X_train.shape[0],3,X_train.shape[1]/3)
X_test=X_test.reshape(X_test.shape[0],3,X_test.shape[1]/3)
X_val=X_val.reshape(X_val.shape[0],3,X_val.shape[1]/3)

In [21]:
def build_cnn(input_var=None):
    network = lasagne.layers.InputLayer(shape=(None,X_train.shape[1], X_train.shape[2]),
                                        input_var=input_var)
    network = lasagne.layers.Conv1DLayer(
        network, num_filters=8, filter_size=50,
        nonlinearity=lasagne.nonlinearities.rectify,
        #W=lasagne.init.GlorotUniform())
        )
    network = lasagne.layers.MaxPool1DLayer(network, pool_size=10)
    #network = lasagne.layers.Conv2DLayer(
    #    network, num_filters=32, filter_size=(10, 1),
    #    nonlinearity=lasagne.nonlinearities.rectify,
    #    W=lasagne.init.GlorotUniform())
    #network = lasagne.layers.MaxPool2DLayer(network, pool_size=(5, 1))
    network = lasagne.layers.DenseLayer(
        lasagne.layers.dropout(network, p=.25),
        num_units=1000,
        nonlinearity=lasagne.nonlinearities.rectify)
    network = lasagne.layers.DenseLayer(
        lasagne.layers.dropout(network, p=.25),
        num_units=5,
        nonlinearity=lasagne.nonlinearities.softmax)

    return network

In [22]:
#%pdb

In [23]:
# Prepare Theano variables for inputs and targets
input_var = T.tensor3('inputs')
target_var = T.ivector('targets')
# Create neural network model
network = build_cnn(input_var)

In [24]:
prediction = lasagne.layers.get_output(network)
loss = lasagne.objectives.multiclass_hinge_loss(prediction, target_var)
loss = loss.mean()

In [25]:
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.nesterov_momentum(
        loss, params, learning_rate=0.0005, momentum=0.9)

In [26]:
test_prediction = lasagne.layers.get_output(network, deterministic=True)
test_loss = lasagne.objectives.multiclass_hinge_loss(test_prediction,
                                                        target_var)
test_loss = test_loss.mean()

In [27]:
test_acc = T.mean(T.eq(T.argmax(test_prediction, axis=1), target_var),
                  dtype=theano.config.floatX)

In [28]:
from tempfile import mkstemp

In [14]:
#tempfile.mkstemp(dir="../tmp")

In [15]:
#import os
#os.environ["THEANO_FLAGS"] = "blas.ldflags=-llapack -lblas -lgfortran"

In [29]:
train_fn =theano.function([input_var, target_var], loss, updates=updates)

In [30]:
val_fn = theano.function([input_var, target_var], [test_loss, test_acc])

In [31]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]



In [32]:
import time
num_epochs = 20
for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(X_train, y_train, 64, shuffle=True):
        inputs, targets = batch
        targets = targets.astype(np.int32)
        train_err += train_fn(inputs, targets)
        train_batches += 1

    # And a full pass over the validation data:
    val_err = 0
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(X_val, y_val, 64, shuffle=False):
        inputs, targets = batch
        targets = targets.astype(np.int32)
        err, acc = val_fn(inputs, targets)
        val_err += err
        val_acc += acc
        val_batches += 1

    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))
    print("  training loss:\t\t{:.6f}".format(train_err / train_batches))
    print("  validation loss:\t\t{:.6f}".format(val_err / val_batches))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

Epoch 1 of 20 took 7.421s
  training loss:		1.009341
  validation loss:		1.003843
  validation accuracy:		47.82 %
Epoch 2 of 20 took 7.432s
  training loss:		1.002673
  validation loss:		1.000537
  validation accuracy:		47.82 %
Epoch 3 of 20 took 7.406s
  training loss:		0.996440
  validation loss:		0.990307
  validation accuracy:		47.86 %
Epoch 4 of 20 took 7.399s
  training loss:		0.980704
  validation loss:		0.981139
  validation accuracy:		47.86 %
Epoch 5 of 20 took 7.391s
  training loss:		0.974691
  validation loss:		0.979998
  validation accuracy:		47.86 %
Epoch 6 of 20 took 7.388s
  training loss:		0.974240
  validation loss:		0.979472
  validation accuracy:		47.24 %
Epoch 7 of 20 took 7.420s
  training loss:		0.972272
  validation loss:		0.977804
  validation accuracy:		47.27 %
Epoch 8 of 20 took 7.392s
  training loss:		0.967397
  validation loss:		0.973999
  validation accuracy:		48.55 %
Epoch 9 of 20 took 7.408s
  training loss:		0.962555
  validation loss:		0.971212
  vali

In [26]:
from sklearn.metrics import accuracy_score
predicted_values = test_prediction.flatten()
print "test loss:", test_loss(predicted_values,y_test)

test loss:

TypeError: 'TensorVariable' object is not callable

In [27]:
predicted_values

Reshape{1}.0